In [1]:
#First load the mosdef36 environment
# This workflow uses signac, and mbuild
# install CSSI-CP2K by cd CSSI-CP2K and then pip install -e .

In [2]:
!cat iodine.py

import sys
sys.setrecursionlimit(10000)

from cssi_cp2k.classes import SIM as sim

mySim = sim.SIM()

mySim.GLOBAL.RUN_TYPE = "MD"
mySim.GLOBAL.PROJECT  = "iodine-liquid"
mySim.GLOBAL.PRINT_LEVEL = "LOW"


#FORCE EVAL SECTION
mySim.FORCE_EVAL.METHOD='QUICKSTEP'
mySim.FORCE_EVAL.STRESS_TENSOR='ANALYTICAL';

mySim.FORCE_EVAL.DFT.BASIS_SET_FILE_NAME='/home/siepmann/singh891/cp2k-6.1.0/data/BASIS_MOLOPT'
mySim.FORCE_EVAL.DFT.POTENTIAL_FILE_NAME='/home/siepmann/singh891/cp2k-6.1.0/data/GTH_POTENTIALS'
mySim.FORCE_EVAL.DFT.CHARGE=0
mySim.FORCE_EVAL.DFT.MULTIPLICITY=1
mySim.FORCE_EVAL.DFT.MGRID.CUTOFF=600
mySim.FORCE_EVAL.DFT.MGRID.REL_CUTOFF=60
mySim.FORCE_EVAL.DFT.MGRID.NGRIDS=4
mySim.FORCE_EVAL.DFT.QS.METHOD='GPW'
mySim.FORCE_EVAL.DFT.QS.EPS_DEFAULT=1E-10
mySim.FORCE_EVAL.DFT.QS.EXTRAPOLATION='ASPC'
mySim.FORCE_EVAL.DFT.POISSON.PERIODIC="XYZ"
mySim.FORCE_EVAL.DFT.PRINT.E_DENSITY_CUBE.SECTION_PARAMETERS="OFF"
mySim.FORCE_EVAL.DFT.SCF.SCF_GUESS='ATOMIC'
mySim.FORCE_EVAL.DFT.SCF.MAX_SCF=30
my

In [3]:
!python iodine.py

{'Date': datetime.datetime(2020, 2, 27, 8, 59, 21, 51292), 'Type': 'Setter', 'Module': 'THERMOSTAT', 'Variable': 'TYPE', 'ErrorMessage': "Invalid option for MD THERMOSTAT TYPE: NOSEDSJGDHSJG. Valid options are: ['AD_LANGEVIN', 'CSVR', 'GLE', 'NOSE']", 'Location': 'SIM/MOTION/MD/THERMOSTAT'}


In [4]:
!cat md.inp

&GLOBAL
  RUN_TYPE     MD
  PROJECT      iodine-liquid
  PRINT_LEVEL  LOW
&END GLOBAL
&MOTION
  &MD
    ENSEMBLE        NVT
    TIMESTEP        0.5
    STEPS           1000000
    TEMPERATURE     409.0
    &THERMOSTAT       
      REGION          MASSIVE 
      &None             
      &END None             
    &END THERMOSTAT       
    &AVERAGES       
    &END AVERAGES       
    &PRINT       
      &ENERGY       
        &EACH       
        &END EACH       
      &END ENERGY       
      &PROGRAM_RUN_INFO       
        &EACH       
        &END EACH       
      &END PROGRAM_RUN_INFO     
    &END PRINT       
  &END MD
 
  &PRINT 
    &FORCES        OFF
      &EACH       
      &END EACH       
    &END FORCES        
    &RESTART_HISTORY        ON
      &EACH       
        MD              500             
      &END EACH       
    &END RESTART_HISTORY        
    &RESTART        ON
      BACKUP_COPIES        3
      &EACH       
        MD              1             
      &

In [1]:
!cat box.py

import mbuild as mb
class I2(mb.Compound):
    def __init__(self):
        super(I2, self).__init__()
        
        iodine1= mb.Particle(pos=[0.0, 0.0, 0.0], name='I')
        iodine2= mb.Particle(pos=[0.8, 0.0, 0.0], name='I')
        self.add([iodine2,iodine1])
        self.add_bond((iodine2,iodine1))

i2 = I2()
i2.energy_minimize(steps=100, algorithm='steep', forcefield='GAFF')

# GIve the user choice to optimize using GAFF or CP2K, If CP2K, then make the input file, something like opt.inp

box = mb.Box(lengths=[1.5115, 1.5115, 1.5115])

box_of_i2= mb.fill_box(compound=i2, n_compounds=64, box=box)
box_of_i2.energy_minimize(steps=100, algorithm='md', forcefield='GAFF')
box_of_i2.save('iodine.xyz')
with open('iodine.xyz', 'r') as fin:
    data = fin.read().splitlines(True)
with open('iodine.xyz', 'w') as fout:
    fout.writelines(data[2:])





In [5]:
!python box.py

/home/rs/anaconda3/envs/mosdef36/lib/python3.6/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/home/rs/anaconda3/envs/mosdef36/lib/python3.6/site-packages/mbuild/compound.py:1798: UserWarning: Performing energy minimization using the Open Babel package. Please refer to the documentation to find the appropriate citations for Open Babel and the GAFF force field
  "Open Babel and the {} force field".format(forcefield))
OBForceField::GenerateVelocities()


In [6]:
!cat iodine.xyz

I    8.882    3.912    4.271
I    8.257    6.028    6.214
I    2.565    7.867   10.974
I    3.768    9.689   12.825
I   10.322    5.403    5.118
I   13.110    6.229    5.381
I    9.704    4.816    2.430
I   11.981    4.295    4.218
I    2.682    0.033    1.681
I    4.947    1.712    0.968
I    5.148    8.601    8.046
I    2.868    9.467    9.547
I    5.017    8.832    0.026
I    5.715    6.909    2.133
I    0.387    5.529    1.228
I    0.377    2.727    0.423
I   10.229    5.566   -0.007
I    9.021    8.170   -0.047
I    3.093    2.012    2.477
I    0.584    0.509    2.427
I    7.658   10.212    0.183
I    9.657   11.379    2.059
I   12.608    0.123    1.907
I   10.582    1.171    0.052
I   10.229    0.596   13.013
I   10.358    3.185   11.692
I    4.428    5.514    8.384
I    2.015    7.055    8.313
I    1.627    4.223    5.677
I    0.068    4.512    3.215
I   12.468    8.154    1.698
I    9.780    7.087    1.751
I    3.981    5.318    6.164
I    4.673    2.514    6.221
I    6.734    

In [7]:
###Signac starts here

In [7]:
!cat iodine.inp

N_MOL L Temp[K] 
48 1.3 409 
48 1.5 673


In [8]:
!python init.py  # initializes the workspace, according to the file iodine.inp, you can look into workspace folder 

In [9]:
!python write_inp.py #writes all the input files needed

HOME: /mnt/c/Users/rstha/Documents/Python Scripts/workflows/cp2k-nvt/workspace/1eb8292acbb5e830124ce6375b8104e7
HOME: /mnt/c/Users/rstha/Documents/Python Scripts/workflows/cp2k-nvt/workspace/942a02a0aa021225638d1c33a8bdbdb2


In [10]:
!python write_xyz.py # creates an initial xyz files

HOME: /mnt/c/Users/rstha/Documents/Python Scripts/workflows/cp2k-nvt/workspace/1eb8292acbb5e830124ce6375b8104e7
/home/rs/anaconda3/envs/mosdef36/lib/python3.6/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/home/rs/anaconda3/envs/mosdef36/lib/python3.6/site-packages/mbuild/compound.py:1798: UserWarning: Performing energy minimization using the Open Babel package. Please refer to the documentation to find the appropriate citations for Open Babel and the GAFF force field
  "Open Babel and the {} force field".format(forcefield))
OBForceField::GenerateVelocities()
HOME: /mnt/c/Users/rstha/Documents/Python Scripts/workflows/cp2k-nvt/workspace/942a02a0aa021225638d1c33a8bdbdb2
/home/rs/anaconda3/envs/mosdef36/lib/python3.6/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, in

In [8]:
! python project.py submit --pretend # this is for the submission on cluster, look into templates/script.sh to change according to your needs

Using environment configuration: UnknownEnvironment
Submitting cluster job 'iodine/5d0a71cf/run_config/0000/c1b9ded9186c314745b62c1ed4c8ccc4':
 - Operation: run_config(5d0a71cf0b09209e7fd73dbdc68e8c0a)


#PBS -l walltime=00:10:00,mem=300gb,nodes=5:ppn=24

module purge
module load mkl
module load fftw
module load intel/cluster/2018

date >> execution.log
set -e
set -u

cd /Users/rs/Documents/research/signac/cp2k/iodine


# run_config(5d0a71cf0b09209e7fd73dbdc68e8c0a)
/Users/rs/anaconda3/envs/mosdef36/bin/python project.py exec run_config 5d0a71cf0b09209e7fd73dbdc68e8c0a

Submitting cluster job 'iodine/9afc385a/run_config/0000/22d7ab14780123f997cbbb613083a805':
 - Operation: run_config(9afc385a608b38eef99c48ee3699f312)


#PBS -l walltime=00:10:00,mem=300gb,nodes=5:ppn=24

module purge
module load mkl
module load fftw
module load intel/cluster/2018

date >> execution.log
set -e
set -u

cd /Users/rs/Documents/research/signac/cp2k/iodine


# run_config(9afc385a608b38eef99c48ee3699f312)
/Use